<a href="https://colab.research.google.com/github/rahiakela/deep-learning-research-and-practice/blob/main/deep-learning-with-python-by-francois-chollet/7-deep-dive-into-keras/01_keras_model_fundamentals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Keras model fundamentals

There are three APIs for building models in Keras:

* The Sequential model, the most approachable API—it’s basically a Python list. As such, it’s limited to simple stacks of layers.
* The Functional API, which focuses on graph-like model architectures. It represents
a nice mid-point between usability and flexibility, and as such, it’s the
most commonly used model-building API.
* Model subclassing, a low-level option where you write everything yourself from
scratch. This is ideal if you want full control over every little thing. However, you
won’t get access to many built-in Keras features, and you will be more at risk of
making mistakes.

<img src='https://github.com/rahiakela/deep-learning-research-and-practice/blob/main/deep-learning-with-python-by-francois-chollet/7-deep-dive-into-keras/images/1.png?raw=1' width='600'/>

##Setup

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import random
import string
import re

import numpy as np

##Sequential model

The simplest way to build a Keras model is to use the Sequential model.

In [2]:
model = keras.Sequential([
    layers.Dense(64, activation="relu"),
    layers.Dense(10, activation="softmax")                      
])

Note that it’s possible to build the same model incrementally via the `add()` method,
which is similar to the `append()` method of a Python list.

In [3]:
model = keras.Sequential()
model.add(layers.Dense(64, activation="relu"))
model.add(layers.Dense(10, activation="softmax"))

As such, the preceding Sequential model does not have any weights, until you actually call it on some data, or call its
method with an input shape `build()`.

In [5]:
# At that point, the model isn’t built yet
# model.weights

```log
ValueError: Weights for model sequential_1 have not yet been created. Weights are created when the Model is first called on inputs or `build()` is called with an `input_shape`.
```

Builds the model—now the model will expect samples of shape (3,). The
None in the input shape signals that the batch size could be anything.

In [6]:
model.build(input_shape=(None, 3))
# Now you can retrieve the model’s weights
model.weights

[<tf.Variable 'dense_2/kernel:0' shape=(3, 64) dtype=float32, numpy=
 array([[ 0.17749679,  0.01807958,  0.20430106,  0.06767303,  0.25197667,
          0.01883757, -0.05213392, -0.2447911 ,  0.11693424, -0.2061596 ,
         -0.17064828, -0.10069327, -0.20086998, -0.05243573, -0.29907587,
          0.08078825, -0.17604849,  0.11384696,  0.19386294,  0.2563058 ,
         -0.06035565, -0.23502591,  0.27104235, -0.27182722, -0.29073983,
          0.01895651, -0.25130033,  0.25646377, -0.02922526,  0.0588116 ,
          0.27234268,  0.09188828,  0.29237998,  0.23397267, -0.25267777,
          0.23168546,  0.25568777,  0.11912537,  0.07228607, -0.00891194,
          0.25923187, -0.2411334 ,  0.0134345 , -0.21463254,  0.09916764,
          0.01390684, -0.22755635, -0.07306112, -0.03803924, -0.11363576,
          0.2759933 ,  0.04954401,  0.18330139, -0.25884604,  0.1439015 ,
          0.07878548, -0.18192309,  0.06084272,  0.26792377, -0.29863173,
         -0.07295774, -0.11770798, -0.27250

After the model is built, you can display its contents via the
method, which `summary()` comes in handy for debugging.

In [7]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 64)                256       
                                                                 
 dense_3 (Dense)             (None, 10)                650       
                                                                 
Total params: 906
Trainable params: 906
Non-trainable params: 0
_________________________________________________________________


As you can see, this model happens to be named “sequential_1.” You can give names
to everything in Keras—every model, every layer.

In [8]:
model = keras.Sequential(name="my_example_model")
model.add(layers.Dense(64, activation="relu", name="my_first_layer"))
model.add(layers.Dense(10, activation="softmax", name="my_last_layer"))

model.build((None, 3))
model.summary()

Model: "my_example_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 my_first_layer (Dense)      (None, 64)                256       
                                                                 
 my_last_layer (Dense)       (None, 10)                650       
                                                                 
Total params: 906
Trainable params: 906
Non-trainable params: 0
_________________________________________________________________


But you can’t print a summary until the model is built! 

There’s actually a way to have your Sequential
built on the fly: just declare the shape of the model’s inputs in advance. You can do this via the Input class.

In [9]:
model = keras.Sequential(name="my_example_model")
model.add(keras.Input(shape=(3, )))
model.add(layers.Dense(64, activation="relu", name="my_first_layer"))

Now you can use `summary()` to follow how the output shape of your model changes as
you add more layers

In [10]:
model.summary()

Model: "my_example_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 my_first_layer (Dense)      (None, 64)                256       
                                                                 
Total params: 256
Trainable params: 256
Non-trainable params: 0
_________________________________________________________________


In [11]:
model.add(layers.Dense(10, activation="softmax", name="my_last_layer"))

model.summary()

Model: "my_example_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 my_first_layer (Dense)      (None, 64)                256       
                                                                 
 my_last_layer (Dense)       (None, 10)                650       
                                                                 
Total params: 906
Trainable params: 906
Non-trainable params: 0
_________________________________________________________________


This is a pretty common debugging workflow when dealing with layers that transform
their inputs in complex ways, such as the convolutional layers.

##Functional API